# GeneriCorp Employee Handbook
This notebook demonstrates how I simulated the documents representing GeneriCorp's employee handbook. I chose NOT to use ChatGPT directly since it would have been too tedious to generate all these by hand! As such, I'll demonstrate for you how we can use the OpenAI API to generate these documents in an automated, efficient manner.

## Notebook Setup

In [21]:
# Importing the necessary Python libraries
import os
import json
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [22]:
# Instantiating the chat model
chat_model = ChatOpenAI(api_key = os.environ['OPENAI_API_KEY'],
                        model = 'gpt-4o-mini')

In [23]:
# Loading the company history from file
with open('../../genericorp/about-gc/company-history.md', 'r', encoding = 'utf-8') as file:
    company_history = file.read()

## Generating Handbook Topics

In [24]:
# Creating the topic generation prompt
TOPIC_GENERATION_PROMPT = '''Imagine a fictional company called GeneriCorp. Their company history is as follows:

{company_history}

Please generate a list of topics they may be found within the employee handbook. Please output them in a Python comma-separated list format. Do not provide any additional text aside from the Python list.'''

# Instantiating the output parser
output_parser = CommaSeparatedListOutputParser()

# Setting up the chat prompt template
topic_generation_prompt_template = PromptTemplate(
    template = TOPIC_GENERATION_PROMPT + '\n{format_instructions}',
    input_variables = ['company_history'],
    partial_variables = {'format_instructions': output_parser.get_format_instructions()}
)

# Creating the topic generation chain
topic_generation_chain = topic_generation_prompt_template | chat_model | output_parser

In [25]:
# Setting the name of a topics cache file
TOPICS_CACHE_FILE = 'handbook_topics.json'

# Checking if cached topics exist
if os.path.exists(TOPICS_CACHE_FILE):
    
    # Loading topics from cache
    with open(TOPICS_CACHE_FILE, 'r') as f:
        handbook_topics = json.load(f)
        
else:

    # Generating topics using the AI model
    handbook_topics = topic_generation_chain.invoke({"company_history": company_history})
    
    # Caching the results
    with open(TOPICS_CACHE_FILE, 'w') as f:
        json.dump(handbook_topics, f)

# Replacing spaces with underscores in handbook topics
handbook_topics = [topic.replace(' ', '-') for topic in handbook_topics]

print(f"Number of topics: {len(handbook_topics)}")
print("Topics:", handbook_topics)

Number of topics: 21
Topics: ['company-culture', 'code-of-conduct', 'benefits-and-compensation', 'employee-wellness-programs', 'diversity-and-inclusion', 'harassment-and-discrimination-policies', 'workplace-safety', 'remote-work-policies', 'performance-reviews', 'training-and-development', 'confidentiality-agreements', 'intellectual-property', 'employee-resources', 'leave-policies', 'travel-policies', 'communication-guidelines', 'ethics-and-compliance', 'work-life-balance', 'grievance-procedures', 'team-building-activities', 'recognition-programs']


## Generating the Handbook Pages from the Topics

In [26]:
# Creating the handbook page generation prompt
HANDBOOK_PAGE_GENERATION_PROMPT = '''Generate a professional and structured employee handbook page on the topic:

{handbook_topic}

As a reminder, here is the company history:

{company_history}

The handbook page should be formatted with clear headings and subheadings, using a formal yet approachable tone suitable for employees.

The page should include the following structured sections:
	1.	Title – A clear and precise title reflecting the topic.
	2.	Introduction – A brief overview explaining why this topic is important and its relevance to employees.
	3.	Policy Overview (if applicable) – A structured breakdown of the rules, guidelines, or best practices employees must follow.
	4.	Employee Responsibilities – Expectations for employees regarding this topic.
	5.	Company Commitment (if applicable) – How the company supports or enforces this policy (e.g., for Diversity & Inclusion, this might include training programs; for Workplace Safety, this might cover compliance measures).
	6.	Procedures – Steps employees need to follow (e.g., how to report an issue, request a benefit, or participate in a program).
	7.	Additional Notes & Compliance – Any legal, ethical, or procedural considerations employees should be aware of.
	8.	Resources & Contact Information – Who employees should reach out to for more information (e.g., HR, department heads, internal portals).

Ensure that the page is concise, clear, and aligned with best HR practices. If applicable, provide bullet points for readability and avoid overly technical language while maintaining professionalism.

Format the output as a well-structured, ready-to-use employee handbook page.
'''

# Setting up the chat prompt template
handbook_page_prompt_template = PromptTemplate(
    template = HANDBOOK_PAGE_GENERATION_PROMPT,
    input_variables = ['handbook_topic', 'company_history'],
)

# Creating the handbook page generation chain
handbook_page_generation_chain = handbook_page_prompt_template | chat_model

In [27]:
# Setting the directory for the employee handbook pages
employee_handbook_dir = '../../genericorp/employee-handbook/'

# Iterating over each of the topics for the employee handbook
for handbook_topic in handbook_topics:
    
    # Defining the file path for the employee handbook page
    file_path = f'{employee_handbook_dir}{handbook_topic}.md'
    
    # Checking if file for the employee handbook page already exists
    if not os.path.exists(file_path):

        # Generating the employee handbook page with the LLM
        response = handbook_page_generation_chain.invoke({"handbook_topic": handbook_topic, "company_history": company_history})
        
        # Saving the employee handbook page to file
        with open(file_path, 'w', encoding = 'utf-8') as file:
            file.write(response.content)